## Archive

In [ ]:
with open('data/GMAN.pkl', 'rb') as f:
	buffer = io.BytesIO(f.read())
model = torch.load(buffer)

/var/folders/9m/r73rtbfx5y57tdt4s_62qsmh0000gn/T/ipykernel_92990/1492739976.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(buffer)


In [ ]:
# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Ensure data is also moved to the same device
trainX, trainTE, trainY = trainX.to(device), trainTE.to(device), trainY.to(device)
valX, valTE, valY = valX.to(device), valTE.to(device), valY.to(device)
testX, testTE, testY = testX.to(device), testTE.to(device), testY.to(device)

trainPred, valPred, testPred = test(args, log)

/Users/darren/Desktop/C4222/GMAN/model/test.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(args.model_file)
/Users/darren/Desktop/C4222/GMAN/model/t

**** testing model ****
loading model from ./data/GMAN.pkl
model restored!
evaluating...


KeyboardInterrupt: 

## Main

In [4]:
def map_arrangement(arr):
    # Initialize the result array with None
    result = [None] * len(arr)

    # Variable to track the next number to assign
    num = 1  # Starting from 1

    # First pass to assign unique numbers
    for i, number in enumerate(arr):
        if result[i] is None:
            result[i] = num  # Assign the current number
            for j in range(i + 1, len(arr)):  # Start from the next index
                if arr[j] == number and result[j] is None:
                    result[j] = num  # Assign the same number to duplicates
            num += 1  # Increment for the next unique number

    return result
                    

# Example usage
import numpy as np

# Create a list of 325 random integers from 0 to 64
np.random.seed(42)  # For reproducibility
arrangement = np.random.randint(0, 65, size=325).tolist()

mapped_result = map_arrangement(arrangement)

print("Original arrangement:", arrangement)
print("Mapped result:", mapped_result)

Original arrangement: [51, 14, 60, 20, 23, 2, 21, 52, 1, 29, 37, 1, 63, 59, 20, 32, 57, 21, 48, 58, 41, 59, 14, 61, 61, 46, 61, 50, 54, 63, 2, 50, 6, 20, 38, 17, 3, 59, 13, 8, 52, 1, 59, 43, 7, 46, 34, 35, 49, 3, 1, 5, 53, 3, 53, 62, 17, 43, 33, 61, 13, 47, 14, 61, 39, 52, 23, 25, 59, 40, 28, 14, 44, 64, 8, 0, 7, 62, 10, 7, 34, 34, 32, 4, 40, 27, 6, 11, 33, 32, 47, 22, 61, 36, 43, 34, 64, 46, 2, 0, 4, 13, 26, 8, 14, 41, 50, 62, 51, 3, 22, 14, 42, 28, 35, 12, 31, 58, 27, 41, 44, 61, 56, 5, 27, 27, 43, 29, 61, 61, 0, 26, 61, 2, 26, 8, 61, 36, 50, 43, 23, 58, 31, 51, 61, 57, 51, 11, 38, 1, 2, 55, 58, 1, 1, 53, 0, 18, 1, 52, 43, 31, 31, 54, 55, 16, 37, 23, 10, 15, 58, 2, 19, 58, 35, 18, 18, 19, 51, 32, 39, 38, 0, 10, 56, 49, 22, 30, 41, 6, 15, 59, 1, 0, 47, 11, 36, 31, 8, 18, 47, 2, 19, 23, 53, 32, 23, 35, 37, 24, 17, 53, 34, 60, 40, 32, 32, 13, 20, 47, 19, 7, 6, 16, 32, 47, 58, 21, 29, 37, 50, 53, 7, 26, 26, 20, 29, 27, 63, 60, 47, 18, 3, 34, 63, 48, 16, 43, 29, 45, 5, 36, 23, 45, 52, 59,

In [2]:
import pandas as pd
import torch
from model.kmeansc import *

import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

import io
import argparse
import time
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
import matplotlib.pyplot as plt

from utils.utils_ import log_string, plot_train_val_loss
from utils.utils_ import count_parameters, load_data

from model.model_ import GMAN
from model.train import train
from model.test import test

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument('--time_slot', type=int, default=5,
                    help='a time step is 5 mins')
parser.add_argument('--num_his', type=int, default=12,
                    help='history steps')
parser.add_argument('--num_pred', type=int, default=12,
                    help='prediction steps')
parser.add_argument('--L', type=int, default=1,
                    help='number of STAtt Blocks')
parser.add_argument('--K', type=int, default=8,
                    help='number of attention heads')
parser.add_argument('--d', type=int, default=8,
                    help='dims of each head attention outputs')
parser.add_argument('--train_ratio', type=float, default=0.7,
                    help='training set [default : 0.7]')
parser.add_argument('--val_ratio', type=float, default=0.1,
                    help='validation set [default : 0.1]')
parser.add_argument('--test_ratio', type=float, default=0.2,
                    help='testing set [default : 0.2]')
parser.add_argument('--batch_size', type=int, default=32,
                    help='batch size')
parser.add_argument('--max_epoch', type=int, default=1,
                    help='epoch to run')
parser.add_argument('--patience', type=int, default=10,
                    help='patience for early stop')
parser.add_argument('--learning_rate', type=float, default=0.001,
                    help='initial learning rate')
parser.add_argument('--decay_epoch', type=int, default=10,
                    help='decay epoch')
parser.add_argument('--traffic_file', default='./data/pems-bay.h5',
                    help='traffic file')
parser.add_argument('--SE_file', default='./data/SE(PeMS).txt',
                    help='spatial embedding file')
parser.add_argument('--model_file', default='./data/GMAN.pkl',
                    help='save the model to disk')
parser.add_argument('--log_file', default='./data/log',
                    help='log file')
args, unknown = parser.parse_known_args()
log = open(args.log_file, 'w')
log_string(log, str(args)[10: -1])
T = 24 * 60 // args.time_slot  # Number of time steps in one day
# load data
log_string(log, 'loading data...')
(trainX, trainTE, trainY, valX, valTE, valY, testX, testTE,
testY, SE, mean, std) = load_data(args)
log_string(log, f'trainX: {trainX.shape}\t\t trainY: {trainY.shape}')
log_string(log, f'valX:   {valX.shape}\t\tvalY:   {valY.shape}')
log_string(log, f'testX:   {testX.shape}\t\ttestY:   {testY.shape}')
log_string(log, f'mean:   {mean:.4f}\t\tstd:   {std:.4f}')
log_string(log, 'data loaded!')
del trainX, trainTE, valX, valTE, testX, testTE, mean, std
# build model
log_string(log, 'compiling model...')

model = GMAN(SE, args, bn_decay=0.1)
loss_criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), args.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer,
                                      step_size=args.decay_epoch,
                                      gamma=0.9)
parameters = count_parameters(model)
log_string(log, 'trainable parameters: {:,}'.format(parameters))

time_slot=5, num_his=12, num_pred=12, L=1, K=8, d=8, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2, batch_size=32, max_epoch=1, patience=10, learning_rate=0.001, decay_epoch=10, traffic_file='./data/pems-bay.h5', SE_file='./data/SE(PeMS).txt', model_file='./data/GMAN.pkl', log_file='./data/log'
loading data...
trainX: torch.Size([36458, 12, 325])		 trainY: torch.Size([36458, 12, 325])
valX:   torch.Size([5189, 12, 325])		valY:   torch.Size([5189, 12, 325])
testX:   torch.Size([10400, 12, 325])		testY:   torch.Size([10400, 12, 325])
mean:   62.7354		std:   9.4388
data loaded!
compiling model...
trainable parameters: 187,463


In [10]:
labels = equal_size_kmeans(SE, 65)

In [11]:
labels

array([ 4,  4, 31, 20,  9, 40, 27, 20, 19, 44, 36, 22, 19, 20, 13, 33, 22,
       20, 62,  9, 27, 40, 20, 13, 62, 44, 39,  8, 30, 35, 13, 39, 33, 61,
       54, 35, 44, 35, 27,  4,  1, 27, 30, 57, 36, 13, 36,  0, 22, 30, 30,
       54, 40, 31, 27, 44, 62, 29, 40, 29, 57, 29, 22,  0, 13, 29, 40, 53,
       46, 52, 29, 52, 45, 38, 52, 28, 28, 36, 57, 28,  0, 33, 39, 57, 44,
       33, 38, 28, 22, 63, 28, 34,  4,  7, 35, 57, 26, 31, 52, 52, 38, 26,
       45, 26, 51, 56, 26, 56, 26, 55, 54, 25, 35, 18, 33, 34, 25, 36, 55,
       25,  6, 62, 54,  6, 55, 59, 62, 25, 23, 49, 54, 25, 55,  6, 45, 24,
       59, 24,  6, 24, 24, 47, 21, 49, 47, 45,  0, 64, 60, 51, 60, 46, 24,
       23, 23, 51, 34, 18, 23, 50, 45, 21, 21, 21, 43, 34, 32, 49, 60, 43,
       59, 59, 19, 18, 19, 64, 17, 17, 17, 17, 17, 60, 43, 18, 64, 63, 53,
        1,  1, 18, 61, 19, 32, 49,  8, 16, 43, 32, 32, 43, 42, 59, 16, 42,
       16, 42, 16, 16, 42, 15, 42, 15, 15, 15, 15, 64, 53, 58,  1, 53, 39,
       39,  9, 14,  7, 14

In [15]:
INPUT = torch.arange(560).reshape(2, 10, 4, 7)
INPUT

tensor([[[[  0,   1,   2,   3,   4,   5,   6],
          [  7,   8,   9,  10,  11,  12,  13],
          [ 14,  15,  16,  17,  18,  19,  20],
          [ 21,  22,  23,  24,  25,  26,  27]],

         [[ 28,  29,  30,  31,  32,  33,  34],
          [ 35,  36,  37,  38,  39,  40,  41],
          [ 42,  43,  44,  45,  46,  47,  48],
          [ 49,  50,  51,  52,  53,  54,  55]],

         [[ 56,  57,  58,  59,  60,  61,  62],
          [ 63,  64,  65,  66,  67,  68,  69],
          [ 70,  71,  72,  73,  74,  75,  76],
          [ 77,  78,  79,  80,  81,  82,  83]],

         [[ 84,  85,  86,  87,  88,  89,  90],
          [ 91,  92,  93,  94,  95,  96,  97],
          [ 98,  99, 100, 101, 102, 103, 104],
          [105, 106, 107, 108, 109, 110, 111]],

         [[112, 113, 114, 115, 116, 117, 118],
          [119, 120, 121, 122, 123, 124, 125],
          [126, 127, 128, 129, 130, 131, 132],
          [133, 134, 135, 136, 137, 138, 139]],

         [[140, 141, 142, 143, 144, 145, 146],
   

In [ ]:
## THIS IS TO CREATE THE MATRIX PERMUTATION 

tensor = torch.zeros((325, 12, 8))
for i in range(tensor.size(0)):
    tensor[i] = i  # Fill the i-th slice with the index value

# Print the tensor
print("Tensor:")
print(tensor)
print(tensor.shape)

# Given grouping tensor
# Define the number of repetitions and the range of values
repetitions = 5
max_value = 65

# Create a tensor with the desired values
grouping = torch.arange(1, max_value + 1).repeat(repetitions)
print(grouping.shape)

# Initialize a dictionary to keep track of counts for each number
count = {}

# Initialize a new tensor to store the transformed values
transformed = torch.empty_like(grouping)

for index, value in enumerate(grouping):
    # If the value is not in the count dictionary, initialize it
    if value.item() not in count:
        count[value.item()] = 0
    
    # Increment the count for this value
    count[value.item()] += 1
    
    # Calculate the new value using the formula
    k = count[value.item()]  # k is the current count of the number
    new_value = 5*(value.item()-1) + k - 1
    
    # Assign the new value to the transformed tensor
    transformed[index] = new_value

# Print the original and transformed tensors
print("Original Tensor:", grouping)
print("Transformed Tensor:", transformed)

final_matrix = tensor[[]]
print(final_matrix)

# Create a matrix of zeros with shape (10, 10)
matrix_size = 325
matrix = torch.zeros((matrix_size, matrix_size), dtype=torch.int)

# Iterate through the tensor and set the corresponding positions to 1
for row, value in enumerate(transformed):
    matrix[row, value] = 1  # Set the (row, value + 1) position to 1

# Print the tensor
print("Tensor:")
print(tensor)

# Print the resulting matrix
print("Resulting Matrix:")
print(matrix)

C = torch.matmul(tensor, matrix.float().t())

print(C)

orig = torch.matmul(C, matrix.float())

print(orig)

Tensor:
tensor([[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         ...,
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.]],

        [[  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         ...,
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         [  2.,   2.,   2.,  ...,   2.,   2.,

RuntimeError: mat1 and mat2 shapes cannot be multiplied (3900x8 and 325x325)

In [ ]:
## THIS IS TO CREATE THE MATRIX PERMUTATION 

tensor = torch.zeros((325, 12, 8))
for i in range(tensor.size(0)):
    tensor[i] = i  # Fill the i-th slice with the index value

# Print the tensor
print("Tensor:")
print(tensor)
print(tensor.shape)

# Given grouping tensor
# Define the number of repetitions and the range of values
repetitions = 5
max_value = 65

# Create a tensor with the desired values
grouping = torch.arange(1, max_value + 1).repeat(repetitions)
print(grouping.shape)

# Initialize a dictionary to keep track of counts for each number
count = {}

# Initialize a new list to store the transformed values
back_to_normal = []  # Create an empty list

for index, value in enumerate(grouping):
    # If the value is not in the count dictionary, initialize it
    if value.item() not in count:
        count[value.item()] = 0
    
    # Increment the count for this value
    count[value.item()] += 1
    
    # Calculate the new value using the formula
    k = count[value.item()]  # k is the current count of the number
    new_value = 5 * (value.item() - 1) + k - 1
    
    # Append the new value to the list
    back_to_normal.append(new_value)
    
# Initialize an empty list to hold the indices
transform = []

# Iterate through each number from 1 to 65
for number in range(1, 66):
    # Get the indices where tensor is equal to the current number
    indices = (grouping == number).nonzero(as_tuple=True)[0]
    
    # Extend the transform list with the indices
    transform.extend(indices.tolist())


# Print the original and transformed tensors
print("Original Tensor:", grouping)
print("Back to Normal List:", back_to_normal)
print("Transformed List:", transform)

print(grouping.shape)
print(len(back_to_normal))
print(len(transform))


final_matrix = tensor[transform]
print(final_matrix)

normal_mat = final_matrix[back_to_normal]
print(normal_mat)



Tensor:
tensor([[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         ...,
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.]],

        [[  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         ...,
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         [  2.,   2.,   2.,  ...,   2.,   2.,